In [1]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/ubuntu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)
CUDA_LAUNCH_BLOCKING=1

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'unk': 1}#Padding indx = 0, unkown_idx = 1, indexing starts from 2
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 2
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                if len(utterance) != 0:
                    utterance_to_idx = []

                    for phone in utterance:
                        if phone not in phone_to_idx:
                            phone = 'unk'

                        utterance_to_idx.append(phone_to_idx[phone])

                    self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.tensor(y_lst)

    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [8]:
def get_domains():
    all_intents = ['increase', 'decrease', 'activate', 'deactivate', 'bring', 'change language']
    return all_intents

def get_intents():
    all_intents = [
        'activate|lamp',
        'activate|lights|bedroom',
        'activate|lights|kitchen',
        'activate|lights|none',
        'activate|lights|washroom',
        'activate|music',
        'bring|juice',
        'bring|newspaper',
        'bring|shoes',
        'bring|socks',
        'change language|Chinese',
        'change language|English',
        'change language|German',
        'change language|Korean',
        'change language|none',
        'deactivate|lamp',
        'deactivate|lights|bedroom',
        'deactivate|lights|kitchen',
        'deactivate|lights|none',
        'deactivate|lights|washroom',
        'deactivate|music',
        'decrease|heat|bedroom',
        'decrease|heat|kitchen',
        'decrease|heat|none',
        'decrease|heat|washroom',
        'decrease|volume',
        'increase|heat|bedroom',
        'increase|heat|kitchen',
        'increase|heat|none',
        'increase|heat|washroom',
        'increase|volume'
        ]

    return all_intents

def get_intent_labels(class_type):
    if class_type == 'domain':
        all_intents = get_domains()
    else:
        all_intents = get_intents()
        
    intent_labels = {}
    labels_to_intents = {}
    for i, intent in enumerate(all_intents):
        intent_labels[intent] = i
        labels_to_intents[i] = intent
        
    return intent_labels, labels_to_intents

In [9]:
class_type = 'domain'
split = 'train'

intent_labels, labels_to_intents = get_intent_labels(class_type)

#Loading data
train_file = '../FSC/fsc_' + class_type + '_' + split + '.pkl'
test_file = '../FSC/fsc_' + class_type + '_test.pkl'
#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
print(len(phone_to_idx))

49


In [10]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [14]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size=50, embed_size=128, hidden_size=512, label_size=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(embed_size*3)

        self.lstm = nn.LSTM(embed_size*3, hidden_size, num_layers=1)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        # B,T,H
        input = self.embed(x)

        # (B,T,H) -> (B,H,T)
        input = input.transpose(1,2)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        # (B,H,T)
        input = F.relu(self.batchnorm(cnn_output))
        #input = F.relu(cnn_output)
        
        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)

        output, (hn, cn) = self.lstm(pack_tensor)

        #output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)

        #output = torch.cat([hn[0], hn[1]], dim=1)
        logits = self.linear(hn[0])

        return logits

In [17]:
model = RNNClassifier()
opt = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
#opt = SGD(model.parameters(), lr=0.05)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(50, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (cnn3): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
  (batchnorm): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(384, 512)
  (linear): Linear(in_features=512, out_features=6, bias=True)
)

In [ ]:
print(class_type, split)
max_acc = 0

for j in range(1000):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()

        logits = model(x, lengths)

        loss = criterion(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
                    

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:
        
        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print(j, "validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

domain train
train acc:  0.6836209506509234  train loss:  0.787785612087882 --time: 27.492029905319214
0 validation:  0.8591772151898734 --max 0.8591772151898734 --time: 31.47409701347351
train acc:  0.864149474503698  train loss:  0.36246624029143737 --time: 27.853115797042847
1 validation:  0.9140295358649789 --max 0.9140295358649789 --time: 31.722460508346558
train acc:  0.9064919337398901  train loss:  0.2502493569781767 --time: 27.731736421585083
2 validation:  0.9332805907172996 --max 0.9332805907172996 --time: 31.641839742660522
train acc:  0.9295013191470957  train loss:  0.19262074617391134 --time: 27.909013986587524
3 validation:  0.9332805907172996 --max 0.9332805907172996 --time: 31.68420958518982
train acc:  0.9482288828337875  train loss:  0.14559737438572704 --time: 27.641254663467407
4 validation:  0.930379746835443 --max 0.9332805907172996 --time: 31.581095695495605
train acc:  0.9633233856667099  train loss:  0.09995953808667252 --time: 27.579711198806763
5 validation

train acc:  0.998572726093162  train loss:  0.0028406720176256824 --time: 27.853556394577026
46 validation:  0.9385548523206751 --max 0.9409282700421941 --time: 31.689669132232666
train acc:  0.9984429739198132  train loss:  0.003086320090558488 --time: 27.856521606445312
47 validation:  0.9375 --max 0.9409282700421941 --time: 31.5884428024292
train acc:  0.9986592275420614  train loss:  0.0027616945907493824 --time: 27.91258692741394
48 validation:  0.9390822784810127 --max 0.9409282700421941 --time: 31.623303651809692
train acc:  0.9983132217464643  train loss:  0.002949312599497646 --time: 28.093059539794922
49 validation:  0.9396097046413502 --max 0.9409282700421941 --time: 31.54794955253601
train acc:  0.9986159768176117  train loss:  0.002930222492520659 --time: 28.29374122619629
50 validation:  0.9385548523206751 --max 0.9409282700421941 --time: 31.87895917892456
train acc:  0.9985294753687124  train loss:  0.0028460596160826018 --time: 28.122591972351074
51 validation:  0.93855

train acc:  0.9987024782665109  train loss:  0.002590599928627843 --time: 27.826766967773438
92 validation:  0.9443565400843882 --max 0.9451476793248945 --time: 31.625261783599854
train acc:  0.9984862246442628  train loss:  0.00262878247550105 --time: 28.033705472946167
93 validation:  0.9448839662447257 --max 0.9451476793248945 --time: 31.718806743621826
train acc:  0.9986592275420614  train loss:  0.002564741048591005 --time: 27.964536905288696
94 validation:  0.945675105485232 --max 0.945675105485232 --time: 31.656715154647827
train acc:  0.9988754811643095  train loss:  0.002492348921966182 --time: 27.879729986190796
95 validation:  0.944620253164557 --max 0.945675105485232 --time: 31.85266613960266
train acc:  0.9986159768176117  train loss:  0.0026224660800810286 --time: 27.88354182243347
96 validation:  0.9435654008438819 --max 0.945675105485232 --time: 31.880082607269287
train acc:  0.9986592275420614  train loss:  0.002500162743998877 --time: 27.525024890899658
97 validation:

In [ ]:
1 layer, 512